In [1]:
import json
import mplleaflet
import matplotlib.pyplot as plt
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from gensim.models import Word2Vec
from tqdm import tqdm
from collections import defaultdict
import copy
import random
Infinity = int(2**31)-1

#Yang Liu,liuy@chalmers.se
#In Python 3, reproducibility between interpreter launches also requires use of the PYTHONHASHSEED environment variable to control hash randomization
#This experiment is based on win11 and the environment variable PYTHONHASHSEED is set to 2022
np.random.seed(2022)
random.seed(2022)

def erp_per_edit(actual,sub,matrix,g=1000):
    '''
    Outputs ERP of comparing sub to actual divided by the number of edits involved
    in the ERP. If there are 0 edits, returns 0 instead.

    Parameters
    ----------
    actual : list
        Actual route.
    sub : list
        Submitted route.
    matrix : dict
        Normalized cost matrix.
    g : int/float, optional
        ERP gap penalty. The default is 1000.

    Returns
    -------
    int/float
        ERP divided by number of ERP edits or 0 if there are 0 edits.

    '''
    total,count=erp_per_edit_helper(actual,sub,matrix,g)
    if count==0:
        return 0
    else:
        return total/count

def erp_per_edit_helper(actual,sub,matrix,g=1000,memo=None):
    '''
    Calculates ERP and counts number of edits in the process.

    Parameters
    ----------
    actual : list
        Actual route.
    sub : list
        Submitted route.
    matrix : dict
        Normalized cost matrix.
    g : int/float, optional
        Gap penalty. The default is 1000.
    memo : dict, optional
        For memoization. The default is None.

    Returns
    -------
    d : float
        ERP from comparing sub to actual.
    count : int
        Number of edits in ERP.

    '''
    if memo==None:
        memo={}
    actual_tuple=tuple(actual)
    sub_tuple=tuple(sub)
    if (actual_tuple,sub_tuple) in memo:
        d,count=memo[(actual_tuple,sub_tuple)]
        return d,count
    if len(sub)==0:
        d=gap_sum(actual,g)
        count=len(actual)
    elif len(actual)==0:
        d=gap_sum(sub,g)
        count=len(sub)
    else:
        head_actual=actual[0]
        head_sub=sub[0]
        rest_actual=actual[1:]
        rest_sub=sub[1:]
        score1,count1=erp_per_edit_helper(rest_actual,rest_sub,matrix,g,memo)
        score2,count2=erp_per_edit_helper(rest_actual,sub,matrix,g,memo)
        score3,count3=erp_per_edit_helper(actual,rest_sub,matrix,g,memo)
        option_1=score1+dist_erp(head_actual,head_sub,matrix,g)
        option_2=score2+dist_erp(head_actual,'gap',matrix,g)
        option_3=score3+dist_erp(head_sub,'gap',matrix,g)
        d=min(option_1,option_2,option_3)
        if d==option_1:
            if head_actual==head_sub:
                count=count1
            else:
                count=count1+1
        elif d==option_2:
            count=count2+1
        else:
            count=count3+1
    memo[(actual_tuple,sub_tuple)]=(d,count)
    return d,count

def normalize_matrix(mat):
    '''
    Normalizes cost matrix.

    Parameters
    ----------
    mat : dict
        Cost matrix.

    Returns
    -------
    new_mat : dict
        Normalized cost matrix.

    '''
    new_mat=mat.copy()
    
    time_list=[]
    for origin in mat:
        for destination in mat[origin]:
            time_list.append(mat[origin][destination])
    avg_time=np.mean(time_list)
    std_time=np.std(time_list)
    min_new_time=np.inf
    
    for origin in mat:
        for destination in mat[origin]:
            old_time=mat[origin][destination]
            new_time=(old_time-avg_time)/std_time
            if new_time<min_new_time:
                min_new_time=new_time
            new_mat[origin][destination]=new_time
            
    for origin in new_mat:
        for destination in new_mat[origin]:
            new_time=new_mat[origin][destination]
            shifted_time=new_time-min_new_time
            new_mat[origin][destination]=shifted_time
            
    return new_mat

def gap_sum(path,g):
    '''
    Calculates ERP between two sequences when at least one is empty.

    Parameters
    ----------
    path : list
        Sequence that is being compared to an empty sequence.
    g : int/float
        Gap penalty.

    Returns
    -------
    res : int/float
        ERP between path and an empty sequence.

    '''
    res=0
    for p in path:
        res+=g
    return res

def dist_erp(p_1,p_2,mat,g=1000):
    '''
    Finds cost between two points. Outputs g if either point is a gap.

    Parameters
    ----------
    p_1 : str
        ID of point.
    p_2 : str
        ID of other point.
    mat : dict
        Normalized cost matrix.
    g : int/float, optional
        Gap penalty. The default is 1000.

    Returns
    -------
    dist : int/float
        Cost of substituting one point for the other.

    '''
    if p_1=='gap' or p_2=='gap':
        dist=g
    else:
        dist=mat[p_1][p_2]
    return dist

def seq_dev(actual,sub):
    '''
    Calculates sequence deviation.

    Parameters
    ----------
    actual : list
        Actual route.
    sub : list
        Submitted route.

    Returns
    -------
    float
        Sequence deviation.

    '''
    actual=actual[1:-1]
    sub=sub[1:-1]
    comp_list=[]
    for i in sub:
        comp_list.append(actual.index(i))
        comp_sum=0
    for ind in range(1,len(comp_list)):
        comp_sum+=abs(comp_list[ind]-comp_list[ind-1])-1
    n=len(actual)
    return (2/(n*(n-1)))*comp_sum

def isinvalid(actual,sub):
    '''
    Checks if submitted route is invalid.

    Parameters
    ----------
    actual : list
        Actual route.
    sub : list
        Submitted route.

    Returns
    -------
    bool
        True if route is invalid. False otherwise.

    '''
    if len(actual)!=len(sub) or set(actual)!=set(sub):
        return True
    elif actual[0]!=sub[0]:
        return True
    else:
        return False

def route2list(route_dict):
    '''
    Translates route from dictionary to list.

    Parameters
    ----------
    route_dict : dict
        Route as a dictionary.

    Returns
    -------
    route_list : list
        Route as a list.

    '''
    if 'proposed' in route_dict:
        stops=route_dict['proposed']
    elif 'actual' in route_dict:
        stops=route_dict['actual']
    route_list=[0]*(len(stops)+1)
    for stop in stops:
        route_list[stops[stop]]=stop
    route_list[-1]=route_list[0]
    return route_list

def get_solution(manager, routing, solution):
    """Prints solution on console."""
    #print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    #plan_output = 'Route for vehicle 0:\n'
    plan_output= []
    #route_distance = 0
    while not routing.IsEnd(index):
        plan_output.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    return plan_output

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return m['distance_matrix'][from_node][to_node]

def check_m_valid(m):
    if m["depot"] < 0:
        return False
    if m["depot"] >= len(m["distance_matrix"]):
        return False
    if len(m["distance_matrix"]) == 0:
        return False
    return True

In [2]:
def load_json(file):
    with open(file,'r') as f:
        output = json.load(f)
    return output

class Data:
    def __init__(self, path='.'):
        
        self.actual_sequences = load_json(f'{path}/actual_sequences.json')
        self.invalid_sequence_scores = load_json(f'{path}/invalid_sequence_scores.json')
        self.package_data = load_json(f'{path}/package_data.json')
        self.route_data = load_json(f'{path}/route_data.json')
        self.travel_times = load_json(f'{path}/travel_times.json')
            
        self.route_id = list(self.travel_times.keys())
        self.all_stations = pd.Series([self.get_station(i) for i in range(len(self))])
        
    def __getitem__(self, idx):
        return self.route_id[idx]
    
    def __len__(self):
        return len(self.route_id)
    
    # get the depot of given route
    def get_depot(self, idx):
        route_id = self[idx]
        depot = [k for k,v in self.route_data[route_id]['stops'].items() if v['type']=='Station'][0]
        return depot
    
    # get station of route
    def get_station(self, idx):
        route_id = self[idx]
        station = self.route_data[route_id]['station_code']
        return station
    
    def get_score(self, idx):
        route_id = self[idx]
        score = self.route_data[route_id]['route_score']
        return score
        
    # get processed sentence for all index
    def get_sentences(self, all_idx):
        all_result = []

        for idx in all_idx:
            route_id = self[idx]
            station = self.get_station(idx)
            stops = [[k,*v.values()] for k,v in self.route_data[route_id]['stops'].items()]
            for i, k in enumerate(self.actual_sequences[route_id]['actual'].values()):
                stops[i].append(k)
            stops.sort(key= lambda x:x[-1])
            old_x = ""
            result = []
            for x in [x[-2] for x in stops if not pd.isna(x[-2])]:
                #x = x.split('.')[0]
                if x!=old_x:
                    result.append(x)
                old_x = x
            result = [station] + result
            all_result.append(result)
        return all_result 
    
    # build data model for ortools
    def build_data_model(self, idx):
        route_id = self[idx]
        m = {}
        #m['distance_matrix'] = (pd.DataFrame(self.travel_times[route_id]).values*10).astype(int)
        m['num_vehicles'] = 1
        depot = self.get_depot(idx)
        depot = list(self.travel_times[route_id].keys()).index(depot)
        distance_matrix = pd.DataFrame(self.travel_times[route_id]).values*10
        dummy_matrix = np.zeros([len(distance_matrix)+1,len(distance_matrix)+1],dtype=int)
        dummy_matrix[-1,:] = Infinity
        dummy_matrix[-1,depot] = 0
        dummy_matrix[-1,-1] = 0
        dummy_matrix[:-1,:-1] = distance_matrix
        m['distance_matrix']  =  dummy_matrix
        m['depot'] = len(dummy_matrix)-1
        return m
    
    def build_data_model_fix_end(self, idx):
        route_id = self[idx]
        m = {}
        m['distance_matrix'] = (pd.DataFrame(self.travel_times[route_id]).values*10).astype(int)
        m['num_vehicles'] = 1
        depot = [k for k,v in self.route_data[route_id]['stops'].items() if v['type']=='Station'][0]
        m['depot'] = list(self.travel_times[route_id].keys()).index(depot)
        return m
    
    def get_sentences_v2(self, all_idx):
        all_result = []
        for idx in all_idx:
            route_id = self[idx]
            station = self.get_station(idx)
            stops = [[k,*v.values()] for k,v in self.route_data[route_id]['stops'].items()]
            for i, k in enumerate(self.actual_sequences[route_id]['actual'].values()):
                stops[i].append(k)
            stops.sort(key= lambda x:x[-1])  
            #result = [station] + list(pd.DataFrame(stops)[1:][4].drop_duplicates().dropna().values)
            
            result=[]
            for x in [x[-2] for x in stops if not pd.isna(x[-2])]:
                if x not in result:
                    result.append(x)
                old_x = x
            result = [station] + result
            
            all_result.append(result)
        return all_result 
    
    def get_sentences_v3(self, all_idx):
        all_result = []
        for idx in all_idx:
                route_id = self[idx]
                station = self.get_station(idx)
                stops = [[k,*v.values()] for k,v in self.route_data[route_id]['stops'].items()]
                for i, k in enumerate(self.actual_sequences[route_id]['actual'].values()):
                    stops[i].append(k)
                stops.sort(key= lambda x:x[-1])  
                score=self.get_score(idx)
                stops=pd.DataFrame(stops)[1:]
                zone_center=stops.groupby(4)[1,2].agg(np.mean).reset_index(drop=True)
                result = [station] + list(round(zone_center[1],2).astype(str)+round(zone_center[2],2).astype(str))
                #temp = [[score]+result[i:i+16] for i in range(len(result)-16+1)]
                all_result.append(result)
        return all_result#[item for sublist in all_result for item in sublist]  
    
  

In [3]:
data = Data('./data')

In [4]:
def get_zone_rank_v2(idx):
    
    route_id = data[idx]
    zone = {}
    station = data.get_station(idx)
    for k,v in data.route_data[data[idx]]['stops'].items():
        if v['type']=='Dropoff':
            if not pd.isna(v['zone_id']):
                zone[k] = v['zone_id']#.split('.')[0]
            else:
                distance_matrix = pd.DataFrame(data.travel_times[data[idx]])
                distance_matrix.loc[k,k]=1000
                real_location=np.argmin(distance_matrix.loc[k,:])
                for i,j in data.route_data[data[idx]]['stops'].items():
                    if i==distance_matrix.loc[k,:].index[real_location]:
                        zone[k] = j['zone_id']
                        break
        else:
            zone[k] = station
    index = data.all_stations[(data.all_stations==station)&(data.all_stations.index!=idx)].index
    sentences = data.get_sentences_v2(index)
    
    unique_zone = set(list(np.unique(list(zone.values()))))
    max_pattern = sentences[np.argmax([len(set(s).intersection(unique_zone)) for s in sentences])]
    rank = defaultdict(lambda:float('nan'))
    rank.update(dict(zip(max_pattern,range(len(max_pattern)))))
    distance = pd.DataFrame(data.travel_times[route_id])
    for k,v in zone.items():
        if v not in rank:
            values = (distance[k]+distance.loc[:,k]).copy()
            values = values.sort_values()            
            for vv in values.index:
                #print(vv)
                if (zone[vv] in rank) & (zone[vv]!=station):
                    zone[k] = zone[vv]
                    break
    return zone,rank

In [5]:
def build_zone_data_model(idx, depot, stops, end):
    route_id = data[idx]
    sequence = [depot]+stops+[end]
    distance_matrix = pd.DataFrame(data.travel_times[route_id])
    distance_matrix = distance_matrix.loc[sequence,sequence].values*10
    
    m = {}
    dummy_matrix = np.zeros([len(distance_matrix)+1,len(distance_matrix)+1],dtype=int)
    dummy_matrix[:-1,:-1] = distance_matrix
    dummy_matrix[-1,:] = Infinity
    dummy_matrix[:,-1] = Infinity
    dummy_matrix[-1,0] = 0
    dummy_matrix[-2,-1] = 0
    dummy_matrix[-1,-1] = 0
    m['num_vehicles'] = 1
    m['distance_matrix']  =  dummy_matrix
    m['depot'] = len(dummy_matrix)-1
    m['sequence'] = sequence
    return m

In [6]:
def build_zone_data_model_last(idx, depot, stops):
    route_id = data[idx]
    sequence = [depot]+stops
    distance_matrix = pd.DataFrame(data.travel_times[route_id])
    distance_matrix = distance_matrix.loc[sequence,sequence].values*10
    
    m = {}
    dummy_matrix = np.zeros([len(distance_matrix)+1,len(distance_matrix)+1],dtype=int)
    dummy_matrix[:-1,:-1] = distance_matrix
    dummy_matrix[-1,:] = Infinity
    dummy_matrix[-1,0] = 0
    dummy_matrix[:,-1] = 0
    dummy_matrix[-1,-1] = 0
    
    m['num_vehicles'] = 1
    m['distance_matrix']  =  dummy_matrix
    m['depot'] = len(dummy_matrix)-1
    m['sequence'] = sequence
    return m

In [7]:
def get_zone_rank_w2v(idx):
    station = data.get_station(idx)
    if station in ['DAU1','DCH3','DCH4','DLA3','DLA4','DLA5','DLA8','DSE4']:
        w_w=8
    elif station in ['DBO1','DBO2','DBO3','DLA7','DLA9']:
        w_w=12
    else:
        w_w=4
    zone = {}
    for k,v in data.route_data[data[idx]]['stops'].items():
        if v['type']=='Dropoff':
            if not pd.isna(v['zone_id']):
                zone[k] = v['zone_id']
            else:
                distance_matrix = pd.DataFrame(data.travel_times[data[idx]])
                distance_matrix.loc[k,k]=1000
                real_location=np.argmin(distance_matrix.loc[k,:])
                for i,j in data.route_data[data[idx]]['stops'].items():
                    if i==distance_matrix.loc[k,:].index[real_location]:
                        zone[k] = j['zone_id']
                        break
        else:
            zone[k] = station
    index = data.all_stations[(data.all_stations==station)&(data.all_stations.index!=idx)].index
    sentences = data.get_sentences_v2(index)   
    unique_zone = list(np.unique(list(zone.values())))
    ###
    count=np.max([len(set(s).intersection(unique_zone)) for s in sentences])
    if count== len(unique_zone):
        #print(idx,count,len(unique_zone))
        max_pattern = sentences[np.argmax([len(set(s).intersection(unique_zone)) for s in sentences])]
        rank = defaultdict(lambda:float('nan'))
        rank.update(dict(zip(max_pattern,range(len(max_pattern)))))
        distance = pd.DataFrame(data.travel_times[data[idx]])
        for k,v in zone.items():
            if v not in rank:
                values = (distance[k]+distance.loc[:,k]).copy()
                values = values.sort_values()            
                for vv in values.index:
                #print(vv)
                    if (zone[vv] in rank) & (zone[vv]!=station):
                        zone[k] = zone[vv]
                        break
    ###
    else:
        #print(idx,count,len(unique_zone))
        sentences_new=[]
        for s in sentences:
            if len(set(s).intersection(unique_zone))>3:
                for i in range(5):
                    sentences_new.append(s)
            else: 
                sentences_new.append(s)
        random.shuffle(sentences_new)
    
        w2v = Word2Vec(sentences=sentences_new, seed=2022,vector_size=30, window=w_w, min_count=1, workers=1, epochs=20,sg=1)       
        x = w2v.wv[unique_zone]

        x = cosine_similarity(x)
        np.fill_diagonal(x, float('-inf'))
        result = [unique_zone.index(station)]
        index = result[-1]
        for _ in range(len(unique_zone)-1):
            x[:,index] = float('-inf')
            index = x[result[-1]].argmax()   
            result.append(index)
    
        rank = dict(zip(np.array(unique_zone)[result],range(len(result))))
    return zone,rank

In [8]:
group=pd.read_csv('./data/group.csv')
group['cold_start']=pd.read_csv('./data/cold_start.csv')['cold_start']

In [9]:
select=group[(group.cold_start==1)].idx

In [ ]:
import time
scores = []
import operator
for idx in tqdm(select):
    or_time=0 #This experiment has tested the running time of different algorithms on i5 cpu, and the running time will be shorter with a better processor

    zone,zone_rank = get_zone_rank_w2v(idx)
    
    route_id = data[idx]
    distance_matrix = pd.DataFrame(data.travel_times[route_id])#.values*10
    zone = pd.DataFrame(zone,index=['zone_id']).T
    zone['zone_rank'] = zone.zone_id.map(zone_rank)
    zone = zone.reset_index()
    zone_rank = zone.zone_rank.dropna().unique()
    zone_rank.sort()
    if len(zone_rank)>1:#If set to 'if len(zone_rank)<-1' then skip our method and execute the TSP model in else   
        result = [zone[zone.zone_rank==0]['index'].iloc[0]]
        for i in range(1,len(zone_rank)-1):
            depot = result[-1]
            stops = list(zone[zone.zone_rank==zone_rank[i]]['index'].values)
            optional_ends = zone[zone.zone_rank==zone_rank[i+1]]['index'].values
            end = distance_matrix.loc[stops,optional_ends].min().sort_values().index[0]
            m = build_zone_data_model(idx, depot, stops, end)
            if check_m_valid(m):
                or_start=time.perf_counter()
                manager = pywrapcp.RoutingIndexManager(len(m['distance_matrix']), m['num_vehicles'], m['depot'])
                routing = pywrapcp.RoutingModel(manager)
                transit_callback_index = routing.RegisterTransitCallback(distance_callback)
                routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
                search_parameters = pywrapcp.DefaultRoutingSearchParameters()
                search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
                solution = routing.SolveWithParameters(search_parameters)
                or_end=time.perf_counter()
                or_time=or_time+or_end-or_start
                solution = get_solution(manager, routing, solution)
                result+=list(np.array(m['sequence'])[solution[2:-1]])
            else:
                result+=m["sequence"][1:-1]

        depot = result[-1]
        stops = list(zone[zone.zone_rank==zone_rank[-1]]['index'].values)

        m = build_zone_data_model_last(idx, depot, stops)
        if check_m_valid(m):
            or_start=time.perf_counter()
            manager = pywrapcp.RoutingIndexManager(len(m['distance_matrix']), m['num_vehicles'], m['depot'])
            routing = pywrapcp.RoutingModel(manager)
            transit_callback_index = routing.RegisterTransitCallback(distance_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
            solution = routing.SolveWithParameters(search_parameters)
            or_end=time.perf_counter()
            or_time=or_time+or_end-or_start
            solution = get_solution(manager, routing, solution)
            result+=list(np.array(m['sequence'])[solution[2:]])
        else:
            result += m["sequence"][1:]
            
        sub = {'proposed':dict(zip(result,range(len(result))))}
    else:#TSP model
        m = data.build_data_model_fix_end(idx)
        if check_m_valid(m):
            or_start=time.perf_counter()
            manager = pywrapcp.RoutingIndexManager(len(m['distance_matrix']), m['num_vehicles'], m['depot'])
            routing = pywrapcp.RoutingModel(manager)
            transit_callback_index = routing.RegisterTransitCallback(distance_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
            solution = routing.SolveWithParameters(search_parameters)
            or_end=time.perf_counter()
            or_time=or_time+or_end-or_start
            solution = get_solution(manager, routing, solution)
        else:
            solution = list(range(len(m["distance_matrix"])))
            solution[m["depot"]] = -1
            
        sub = {'proposed':dict(zip(data.route_data[data[idx]]['stops'].keys(),np.array(solution).argsort()))}
    sub = route2list(sub)
    actual = data.actual_sequences[data[idx]]
    actual = route2list(actual)
    matrix = copy.deepcopy(data.travel_times[data[idx]])
    matrix = normalize_matrix(matrix)
    scores.append([idx,seq_dev(actual,sub),erp_per_edit(actual,sub,matrix,1000)])

In [ ]:
scores=pd.DataFrame(scores,columns=['idx','sd','erp','runtime'])
select=group[(group.cold_start==1)].reset_index(drop=True)
select['sd']=round(scores['sd'],3)
select['erp']=round(scores['erp'],3)
select['result_w2v']=round(scores['sd']*scores['erp'],3)
select['runtime']=round(scores['runtime'],5)
select.to_csv('result.csv',index=False)